In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.metrics import accuracy_score
%matplotlib inline

In [2]:
df = pd.read_csv("pay_pricedayu0.csv")
df["leibie"] = 0
df["leibie"] = df.apply(lambda x: 1 if ((-x["pay_price"] + x["prediction_pay_price"])>=15 and  x["pay_price"] != 0) else x["leibie"] ,axis=1)
df["leibie"] = df.apply(lambda x: 2 if x["pay_price"] == x["prediction_pay_price"] else x["leibie"],axis=1)
df["leibie"] = df.apply(lambda x: 3 if ((-x["pay_price"]+x["prediction_pay_price"])<15 and  x["pay_price"] != 0 and (-x["pay_price"]+x["prediction_pay_price"])>0) else x["leibie"],axis=1)
df_ = df.drop(['prediction_pay_price'],axis=1)
dt1=pd.to_datetime(df_["register_time"])
df_["register_time"] = dt1.dt.dayofyear
df_.shape,df_.columns,df.shape,df.columns

((41439, 110),
 Index([u'user_id', u'register_time', u'wood_add_value', u'wood_reduce_value',
        u'stone_add_value', u'stone_reduce_value', u'ivory_add_value',
        u'ivory_reduce_value', u'meat_add_value', u'meat_reduce_value',
        ...
        u'pvp_lanch_count', u'pvp_win_count', u'pve_battle_count',
        u'pve_lanch_count', u'pve_win_count', u'avg_online_minutes',
        u'pay_price', u'pay_count', u'lable', u'leibie'],
       dtype='object', length=110),
 (41439, 111),
 Index([u'user_id', u'register_time', u'wood_add_value', u'wood_reduce_value',
        u'stone_add_value', u'stone_reduce_value', u'ivory_add_value',
        u'ivory_reduce_value', u'meat_add_value', u'meat_reduce_value',
        ...
        u'pvp_win_count', u'pve_battle_count', u'pve_lanch_count',
        u'pve_win_count', u'avg_online_minutes', u'pay_price', u'pay_count',
        u'prediction_pay_price', u'lable', u'leibie'],
       dtype='object', length=111))

In [3]:

# def feature(df):
#     df["wood"] = df["wood_reduce_value"]/(df["wood_add_value"]+0.1)
#     df["meat"] = df["meat_reduce_value"]/(df["meat_add_value"]+0.1)
#     df["ivory"] = df["ivory_reduce_value"]/(df["ivory_add_value"]+0.1)
#     df["stone"] = df["stone_reduce_value"]/(df["stone_add_value"]+0.1)
#     df["magic"] = df["magic_reduce_value"]/(df["magic_add_value"]+0.1)    
    
#     df["general_acceleration"] = df["general_acceleration_reduce_value"]/(df["general_acceleration_add_value"]+0.1)
#     df["building_acceleration"] = df["building_acceleration_reduce_value"]/(df["building_acceleration_add_value"]+0.1)
    
#     df["reaserch_acceleration"] = df["reaserch_acceleration_reduce_value"]/(df["reaserch_acceleration_add_value"]+0.1)
#     df["training_acceleration"] = df["training_acceleration_reduce_value"]/(df["training_acceleration_add_value"]+0.1)
    
    
#     df["sr_outpost"] =df["sr_outpost_tier_3_level"]+df["sr_outpost_tier_2_level"]+df["sr_outpost_tier_4_level"]
#     df["pvp_battle"] = df["pvp_battle_count"]*df["pay_price"]/df["avg_online_minutes"]/df["pay_count"]

    
#     df = df.drop([
#                     'stone_reduce_value',    
#                     ], axis=1)
    
#     return df
# df = df.iloc[:,1:]
# df = feature(df)



In [4]:
cols = [x for x in df.columns if x not in ["user_id", 'register_time',"leibie","prediction_pay_price","lable"]]
train_x = df[cols]
train_y = df.iloc[:,110:]
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.4, random_state=2008)

In [5]:
# X_test = pd.read_csv("test_pay_pricedayu0.csv")
# x_test = X_test[cols]

In [6]:
from sklearn.model_selection import StratifiedKFold

def stacking(x_train, y_train, x_test, cv):

#     le = LabelEncoder()
#     y_train = le.fit_transform(y_train)

    clfs = [
        xgb.XGBClassifier(max_depth = 5,booster= 'gbtree',objective='multi:softmax',gamma=0.1,n_estimators=300),
        lgb.LGBMClassifier(max_depth = 5,objective='multi:softmax',n_estimators=300),
        RandomForestClassifier(max_depth = 6,n_estimators=20),
        GradientBoostingClassifier(max_depth = 5,n_estimators= 200),
    ]

    skf = StratifiedKFold(n_splits=cv)
    skf_dataset = list(skf.split(x_train, y_train))

    # y_count: the kinds of labels
    y_count = len(set(y_train.leibie))
#     print y_count
    
    # blend_train is the probabilities that every clf predicts every label (i.e. y) for every sample
    # it is used to train the clfs in the second layer
    blend_train = np.zeros((x_train.shape[0], len(clfs) * y_count))
    blend_train = pd.DataFrame(blend_train)

    # blend_test is used as the input of the clfs in the second layer to predict the labels of x_test
    blend_test = np.zeros((x_test.shape[0], len(clfs) * y_count))
    blend_test = pd.DataFrame(blend_test)

    for j, clf in enumerate(clfs):
        print j, clf

        # blend_test_j: the probabilities that j-th clf predicts every label for x_test
        blend_test_j = np.zeros((x_test.shape[0], len(skf_dataset) * y_count))
        blend_test_j = pd.DataFrame(blend_test_j)
        
        for k, (train_idx, test_idx) in enumerate(skf_dataset):
            x_train_k = x_train.iloc[train_idx,:]
            y_train_k = y_train.iloc[train_idx,:]
            x_train_holdout = x_train.iloc[test_idx,:]

            clf.fit(x_train_k, y_train_k)
            x_train_holdout_ = clf.predict_proba(x_train_holdout)
            
            #blend_train[test_idx, j*y_count: (j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_train.iloc[test_idx,j*y_count:(j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_test_j.iloc[:, k*y_count: (k+1)*y_count] = clf.predict_proba(x_test)

        # because there are len(skf_dataset) blend_test_j for x_test, it needs to calculated the mean value
        blend_test_j_mean = np.zeros((x_test.shape[0], y_count))
        blend_test_j_mean = pd.DataFrame(blend_test_j_mean)

        # indices: supposed y_count = 3, indices would be [0, 3, 6]
        # it is used to find the corresponding probabilities of the same label, and calculate the mean
        indices = np.arange(len(skf_dataset)) * y_count
        for c in xrange(y_count):
            blend_test_j_mean.iloc[:,c] = pd.DataFrame(blend_test_j.iloc[:, indices].mean(1))
            indices += 1
        blend_test[range(j*y_count,(j+1)*y_count)] = blend_test_j_mean
#         print blend_test_j_mean.head()
#         print blend_test.head()

    clf = LogisticRegression()
    clf.fit(blend_train, y_train)
    y_pred = clf.predict(blend_test)
#     y_pred = le.inverse_transform(y_pred)

    return y_pred#blend_train,blend_test

In [7]:
y_test_pred = stacking(X_train,y_train,X_test,4)
# print u'stacking：', accuracy_score(y_test, y_test_pred)
# df.iloc[:,110:]


# stacking_train['user_id'] = df['user_id']
# stacking_test['user_id'] = X_test['user_id']
# stacking_train['lable'] = df['lable']
# # stacking_train.to_csv('stacking_train12.csv',index=None)
# # stacking_test.to_csv('stacking_test12.csv',index=None)

# stacking_train12 = stacking_train
# stacking_test12 = stacking_test
# stacking_train12_ = stacking_train12.iloc[:,:8]
# stacking_test12_ = stacking_test12.iloc[:,:8]
# # stacking_train12_.columns,stacking_test12_.columns
# lr = LogisticRegression()
# lr.fit(stacking_train12_, stacking_train12.lable)
# y_pred = lr.predict(stacking_test12_)
# y_pred = pd.DataFrame(y_pred)
# stacking_test12['lable'] = y_pred
# stacking_test12__ =stacking_test12.iloc[:,[8,9]]

# stacking_test12__['lable'] = stacking_test12__.apply(lambda x: 2 if(x['lable'] == 0) else 1,axis=1)   
# stacking_test12__.to_csv('stacking_test12_lr.csv',index=None)

0 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=5, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=31, objective='multi:softmax',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
2 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


3 GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
print u'stacking：', accuracy_score(y_test, y_test_pred)

stacking： 0.7891529922779923


In [15]:
y_test['pre'] = y_test_pred
lable1 = y_test[(y_test['leibie']==1)]
lable11 = y_test[(y_test['pre'] ==1) & (y_test['leibie']==1)]
print lable1.shape,lable11.shape
lable2 = y_test[(y_test['leibie']==2)]
lable22 = y_test[(y_test['pre'] ==2) & (y_test['leibie']==2)]
print lable2.shape,lable22.shape
lable3 = y_test[(y_test['leibie']==3)]
lable33 = y_test[(y_test['pre'] ==3) & (y_test['leibie']==3)]
print lable3.shape,lable33.shape

(2684, 2) (1564, 2)
(12098, 2) (11509, 2)
(1794, 2) (8, 2)


F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_test_ = pd.read_csv("test_pay_pricedayu0.csv")
df_test = df_test_[cols]
y_test_pred = stacking(train_x,train_y,df_test,4)

0 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
1 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=5, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=31, objective='multi:softmax',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
2 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_sp

F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


3 GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [22]:
y_test_pred = pd.DataFrame(y_test_pred)
y_test_pred.columns = ['pre_lable']
lable1 = y_test_pred[(y_test_pred['pre_lable']==1)]
print lable1.shape
lable2 = y_test_pred[(y_test_pred['pre_lable']==2)]
print lable2.shape
lable3 = y_test_pred[(y_test_pred['pre_lable']==3)]
print lable3.shape

(3050, 1)
(16495, 1)
(4, 1)
